# 3th_hometask

In [1]:
import numpy as np

## 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

### Исходная функция:

In [2]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [3]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [4]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_6868/851670199.py:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_6868/851670199.py:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

In [5]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0])
calc_logloss(y1, y_pred1)

C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_6868/851670199.py:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\SERGEY~1\AppData\Local\Temp/ipykernel_6868/851670199.py:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

##### Исходная фукция не работает при спрогнозированных вероятностях 1 или 0, так как в этом случае под логарифмом оказывается нуль.

### Изменённая функция:

##### Попробуем добавить какое-нибудь маленькое значение, которое не будет оказывать существенного влияния на метрику, но при этом позволит избежать возникновения нуля под логарифмом.

In [6]:
def calc_logloss(y, y_pred):
    anti_error = 1e-15
    err = - np.mean(y * np.log(y_pred + anti_error) + (1.0 - y) * np.log(1.0 - y_pred + anti_error))
    return err

In [7]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.1642520334860168

##### В сравнении с исходной функцией мы можем наблюдать крайне незначительные изменения метрики, погрешность наблюдается после 14-го знака после запятой.

In [8]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891203

In [9]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0])
calc_logloss(y1, y_pred1)

0.11157177565710368

In [10]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0])
calc_logloss(y1, y_pred1)

-1.110223024625156e-15

##### Попробуем избавиться от отрицательных значений в случае идеального прогноза добавив округление после 14-го знака. Также возьмём модуль ответа, чтобы избежать ответа "-0.0".

In [11]:
def calc_logloss(y, y_pred):
    anti_error = 1e-15
    err = - np.mean(y * np.log(y_pred + anti_error) + (1.0 - y) * np.log(1.0 - y_pred + anti_error))
    return np.abs(np.round(err, 14))

In [12]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891

In [13]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0])
calc_logloss(y1, y_pred1)

0.1115717756571

In [14]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0])
calc_logloss(y1, y_pred1)

0.0

##### Работает!

##### Также можно было бы реализовать отдельный алгоритм для разбора ситуаций, когда модель выдаёт вероятность 0 или 1, но данная реализация выглядит проще.

## 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [15]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [16]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [17]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations + 1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [18]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [19]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

### Исходные аргументы:

In [20]:
W = eval_model(X_st, y, iterations=500, eta=1e-4)

0 [ 0.49667621 -0.13840939  0.6476858   1.52297324] 1.17859583443561
50 [ 0.494784   -0.14564801  0.6475462   1.52014828] 1.16579857492553
100 [ 0.49290109 -0.15285535  0.64740132  1.51733474] 1.15311126857083
150 [ 0.49102761 -0.16003088  0.64725118  1.51453281] 1.14053527533049
200 [ 0.48916364 -0.16717404  0.64709581  1.51174267] 1.12807193269173
250 [ 0.48730929 -0.17428428  0.64693524  1.50896452] 1.11572255659606
300 [ 0.48546465 -0.18136107  0.64676951  1.50619853] 1.10348844262243
350 [ 0.48362982 -0.18840385  0.64659868  1.5034449 ] 1.09137086741919
400 [ 0.48180488 -0.19541206  0.64642281  1.50070383] 1.07937109037212
450 [ 0.47998993 -0.20238516  0.64624195  1.49797551] 1.06749035549159
500 [ 0.47818506 -0.20932258  0.64605619  1.49526014] 1.05572989349779


##### Функция потерь уменьшается вплоть до последнец итерации. Вероятно, алгоритму не хватило итераций для дальнейшего уменьшения функционала ошибки.

### Новые аргументы:

In [21]:
%%time

W = eval_model(X_st, y, iterations=1000000, eta=10)

0 [ -3.29715467 -14.6470242    0.37388435  -4.13843283] 1.17859583443561
100000 [-222.7923953   -15.71449676  -66.00850597  170.47520542] 0.00675437110364
200000 [-266.78113524  -18.4559948   -79.34691143  203.4510617 ] 0.00342406608172
300000 [-292.80097774  -20.07957263  -87.23494243  222.96058506] 0.00228641162731
400000 [-311.30854942  -21.23495797  -92.84508527  236.83857122] 0.00171457701332
500000 [-325.6740421   -22.13199747  -97.19943724  247.61107309] 0.00137097835065
600000 [-337.41292046  -22.8651414  -100.75752022  256.41415326] 0.00114185103385
700000 [-347.33710532  -23.4850202  -103.76550054  263.85652165] 0.00097821593245
800000 [-355.93234768  -24.02193589 -106.37064289  270.30237596] 0.00085553076405
900000 [-363.51236712  -24.49546401 -108.6680526   275.9869416 ] 0.00076014671899
1000000 [-370.29152644  -24.91898307 -110.72271217  281.07095209] 0.00068387064698
Wall time: 30.4 s


##### Увеличение количества итераций на три порядка и скорости обучения на пять порядков позволило существенно сократить значение функционала ошибки.

## 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [22]:
def calc_pred_proba(W, X):
    return sigmoid(X@W)

In [23]:
np.round(calc_pred_proba(W, X_st), 9)

array([3.41305000e-03, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       9.99821475e-01, 0.00000000e+00, 1.00000000e+00, 3.29700000e-05,
       9.96796841e-01, 1.00000000e+00])

In [24]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

##### Результат предсказания довольно близок к истинным ответам.

## 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [25]:
def calc_pred(W, X):
    return np.round(sigmoid(X@W))

In [26]:
calc_pred(W, X_st)

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [27]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

## 5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [28]:
y_pred = calc_pred(W, X_st)

In [29]:
def classification_report(y_true, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    
    for i in range(len(y_true)):
        if (y_true[i] == 1) & (y_pred[i] == y_true[i]):
            TP += 1
        elif (y_true[i] == 0) & (y_pred[i] == y_true[i]):
            TN += 1
        elif (y_true[i] == 1) & (y_pred[i] != y_true[i]):
            FN += 1
        elif (y_true[i] == 0) & (y_pred[i] != y_true[i]):
            FP += 1
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision_1 = TP / (TP + FP)
    precision_0 = TN / (TN + FN)
    recall_1 = TP / (TP + FN)
    recall_0 = TN / (TN + FP)
    f1_score_1 = 2 * precision_1 * recall_1 / (precision_1 + recall_1)
    f1_score_0 = 2 * precision_0 * recall_0 / (precision_0 + recall_0)
    
    print(f"\t\t=== Classification report ===\n" \
          f"\n" \
          f"\t\t\tActual 1\t\tActual 0\n" \
          f"Predicted 1\t\t{TP}\t\t\t{FP}\n" \
          f"Predicted 0\t\t{FN}\t\t\t{TN}\n" \
          f"\n" \
          f"Precision:\t\t{precision_1}\t\t\t{precision_0}\n" \
          f"Recall:\t\t\t{recall_1}\t\t\t{recall_0}\n" \
          f"\n" \
          f"Accuracy:\t\t{accuracy}\n" \
          f"F1-score:\t\t{f1_score_1}\t\t\t{f1_score_0}\n" \
          f"\n" \
          f"Made by SergeyZ06\n" \
          f":D" \
         )            

In [30]:
classification_report(y, y_pred)

		=== Classification report ===

			Actual 1		Actual 0
Predicted 1		5			0
Predicted 0		0			5

Precision:		1.0			1.0
Recall:			1.0			1.0

Accuracy:		1.0
F1-score:		1.0			1.0

Made by SergeyZ06
:D


## 6. Могла ли модель переобучиться? Почему?

##### Линейная логистическая регрессия сремится разделить гиперпространство признаков объектов таким образом, чтобы объекты были корректно разделены на классы, а также максимально удалены от гиперплоскости разделения для максимизации уверенности классификации.

##### В данном случае переобучение не уместно, так как модель пытается выполнить разделение с максимальным удалением каждого признака от гиперплоскости разделения.